In [36]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math

In [37]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [38]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [39]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [40]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [41]:
airnb_searches_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="airbnb_searches", properties=jdbc_properties))

In [42]:
airnb_searches_df.show()

+-------+-------------+--------------------+
|user_id|date_searched|   filter_room_types|
+-------+-------------+--------------------+
|      1|   2022-01-01|entire home,priva...|
|      2|   2022-01-02|entire home,share...|
|      3|   2022-01-02|private room,shar...|
|      4|   2022-01-03|        private room|
+-------+-------------+--------------------+



In [43]:
airnb_searches_df = airnb_searches_df.withColumn('value', explode(split(airnb_searches_df['filter_room_types'], ',')))


In [47]:
airnb_searches_df.show(truncate = False)

+-------+-------------+------------------------+------------+
|user_id|date_searched|filter_room_types       |value       |
+-------+-------------+------------------------+------------+
|1      |2022-01-01   |entire home,private room|entire home |
|1      |2022-01-01   |entire home,private room|private room|
|2      |2022-01-02   |entire home,shared room |entire home |
|2      |2022-01-02   |entire home,shared room |shared room |
|3      |2022-01-02   |private room,shared room|private room|
|3      |2022-01-02   |private room,shared room|shared room |
|4      |2022-01-03   |private room            |private room|
+-------+-------------+------------------------+------------+



In [56]:
airnb_searches_df.groupBy("value").count()\
.withColumnRenamed("value", "Room_Types")\
.withColumnRenamed("count", "Room_count")\
.show()

+------------+----------+
|  Room_Types|Room_count|
+------------+----------+
|private room|         3|
| entire home|         2|
| shared room|         2|
+------------+----------+

